# Step [1]: Prepare libraries and data

## [1.1] Include important libraries

In [1]:
import pandas as pd
import nltk
import numpy
import matplotlib.pyplot as plt            # library for visualization
import matplotlib
import re
import numpy as np
from collections import Counter
from nltk import ngrams
from nltk.tokenize import TweetTokenizer
from nltk.stem.isri import ISRIStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
%matplotlib inline 


## [1.2] Download data
Arabic opinions tweets collected from twitter on covid-19 vaccination

In [3]:
!wget 'https://drive.google.com/uc?export=download&id=1KepfzAhJ7dloG8XaWQf0ovQipDHYS8aI' -O 'final_data.zip'

--2021-12-18 13:24:09--  https://drive.google.com/uc?export=download&id=1KepfzAhJ7dloG8XaWQf0ovQipDHYS8aI
Resolving drive.google.com (drive.google.com)... 172.217.13.78, 2607:f8b0:4004:808::200e
Connecting to drive.google.com (drive.google.com)|172.217.13.78|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-00-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/39j57sqqkuffbd3e0t6ahrgsfcefo7cj/1639833825000/04260309330816471542/*/1KepfzAhJ7dloG8XaWQf0ovQipDHYS8aI?e=download [following]
--2021-12-18 13:24:12--  https://doc-0o-00-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/39j57sqqkuffbd3e0t6ahrgsfcefo7cj/1639833825000/04260309330816471542/*/1KepfzAhJ7dloG8XaWQf0ovQipDHYS8aI?e=download
Resolving doc-0o-00-docs.googleusercontent.com (doc-0o-00-docs.googleusercontent.com)... 142.251.45.97, 2607:f8b0:4004:83f::2001
Connecting to doc-0o-00-docs.googleusercontent.com (doc-0o-00-docs.goo

In [4]:
!unzip final_data.zip

Archive:  final_data.zip
  inflating: test.csv                
  inflating: train.csv               
  inflating: valid.csv               


## [1.3] read data from csv file

In [5]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
valid = pd.read_csv("valid.csv")
train['tweet'][4597]=" "
train['tweet'][4597]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


' '

# Data Splitting

In [6]:
train1 = []
y_train = []
test1 = []
y_test = []
valid1 = []
y_valid = []

for i in  range(len(train)):
    if train['label'][i] == 0:
        y_train.append(train['label'][i])
        train1.append(train['tweet'][i])
    if train['label'][i] == 1:
        y_train.append(train['label'][i])
        train1.append(train['tweet'][i])
for i in range(len(test)):
    if test['label'][i] == 0:
        y_test.append(test['label'][i])
        test1.append(test['tweet'][i])
    if test['label'][i] == 1:
        y_test.append(test['label'][i])
        test1.append(test['tweet'][i])
for i in range(len(valid)):
    if valid['label'][i] == 0:
        y_valid.append(valid['label'][i])
        valid1.append(valid['tweet'][i])
    if valid['label'][i] == 1:
        y_valid.append(valid['label'][i])
        valid1.append(valid['tweet'][i])
count=0
newy=[]
for i,d in enumerate(y_train):
    if d==1:
        if count >2783:
            train1.pop(i-2783)
            continue
        else:   
            newy.append(d)
            count=count+1
    else:
        newy.append(d)
# X_test =np.array(test1)
# X_train=np.array(train1)
# X_valid=np.array(valid1)
y_train=newy
len(y_train)

5567

# perprocessing methods

In [15]:
def delinks_mintions(data,work=1):
    text = []
    if work==1:
        for d in data:
            text.append(re.sub(r'http\S+|@[A-Za-z0-9]+|[A-Za-z0-9]+|[A-Za-z0-9]', '', d))
    if work == 0:
        text.append(data)
        return data
    return text

In [16]:
def derepeted(data,work=1):
    text = []
    if work==1:

        for d in data:
            text.append( re.sub(r"(.)\1+",r"\1",d))
    if work == 0:
        text.append(data)
        return data
    return text


In [17]:
def standardization_of_words(words,replace,tweets,work=1):
    if work ==1 :
        for i in range(0, len(words)):
            tweets = tweets.replace(words[i], replace[i])
    return tweets
def character_standardization(data,work=1):
    text = []
    if work ==1 :
        letter = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ي","\\",'\n', '\t','&quot;','?','؟','!']
        replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ى","",' ', ' ',' ',' ? ',' ؟ ',' ! '] 
        for d in data:
            tweet = re.sub(r'[\u0617-\u061A\u064B-\u0652]',"", d)
            tweet=standardization_of_words(letter,replace,d)
            text.append(tweet)
    if work == 0:
        text.append(data)
        return data
    return text


In [18]:
def dedigits(data,work=1):
    text = []
    if work==1:
        
        for d in data:
            text.append( re.sub(r"\d+","",d))
    if work == 0:
        text.append(data)
        return data
    return text


In [19]:
def denoncharachters(data,work=1):
    text = []
    if work==1:
        
        for d in data:
            text.append( re.sub(r"\W"," ",d))
    if work == 0:
        text.append(data)
        return data
    return text


In [20]:
def deEmoji(data,work=1):
    text = []
    if work==1:
        
        for d in data:
            needed_words= []
            regrex_pattern = re.compile(pattern = "["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   "]+", flags = re.UNICODE)
            text.append(regrex_pattern.sub(r'',d))
    if work == 0:
        text.append(data)
        return data
    return text


In [21]:
# nltk.download()
stopwords_list = nltk.corpus.stopwords.words('arabic')
y= 'كل كلمة كم منذ سنة'

def destopwords(data,work=1):
    text = []
    if work==1:
        stopwords_list = nltk.corpus.stopwords.words('arabic')
        
        for d in data:
            d1=d.split()
            needed_words= []
            for w in d1:
                if w not in (stopwords_list):
                    needed_words.append(w)
            filtered_sentence = " ".join(needed_words)
            text.append( filtered_sentence)
    if work == 0:
        text.append(data)
        return data
    return text


In [22]:
def denoncharachters(data,work=1):
    text = []
    if work==1:
        
        for d in data:
            t1=re.sub(r"\W"," ",d)
            t1=re.sub(r"[A-Za-z]"," ",t1)
            text.append( re.sub(r"\s+"," ",t1))
    if work == 0:
        text.append(data)
        return data
    return text


In [23]:
def stemming(tweet,work=1):
    
    result = []
    text = []
    words = ''
    if work ==1 :
        for d in tweet:
            result = []
            words = d.split()
            stemmer = ISRIStemmer()
            for word in words:
                word = stemmer.norm(word, num=1) 
                if word.startswith('#'):
                    result.append(word)
                    continue
                if not word in stemmer.stop_words:    
                    word = stemmer.pre32(word)         
                    word = stemmer.suf32(word)        
                result.append(word)
        
            re = ' '.join(result)
            text.append(re)
    
    if work == 0:
        text.append(tweet)
        return tweet
    
    return text


In [24]:
def standardization_of_words(words,replace,tweets,work=1):
    if work ==1 :
        for i in range(0, len(words)):
            tweets = tweets.replace(words[i], replace[i])
    return tweets
def character_standardization(tweet,work=1):
    text = []
    if work == 0:
        text.append(tweet)
        return tweet
    if work ==1 :
        letter = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ي","\\",'\n', '\t','&quot;','?','؟','!']
        replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ى","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
        for d in tweet:
            d = re.sub(r'[\u0617-\u061A\u064B-\u0652]',"", d)
            d=standardization_of_words(letter,replace,d)
            text.append(d)
    return text


In [25]:
def preprocess(tweet, flags=[0,0,1,0,0,0,1,0]):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
        flags: list of "work" values for all functions will called here.
    Output:
        cleaned_tweet: tweet after apply all cleaning and normlizaing functions

    """
    tweet = delinks_mintions(tweet, flags[0])
    tweet = derepeted(tweet, flags[1])
    tweet = dedigits(tweet, flags[2])
    tweet = denoncharachters(tweet, flags[3])
    tweet = deEmoji(tweet, flags[4])
    tweet = destopwords(tweet, flags[5])
    tweet = character_standardization(tweet, flags[6])
    tweet = stemming(tweet, flags[7])
    return tweet

## [1.4] Prapere The Comparison Dictionary

In [26]:
model_comparison_table = {}

In [27]:
model_comparison_table['model_name'] = []
model_comparison_table['preprocessing_methods'] = []
model_comparison_table['accuracy'] = []

# Step [2]: Build Baseline

## [2.1] Extract Bag of Words Features

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
# code to be written with students in class
vectorizer = CountVectorizer()
vectorizer.fit(train1)
X_train = vectorizer.transform(train1)
X_valid = vectorizer.transform(valid1)
X_test  = vectorizer.transform(test1)
# X_test =np.array(X_test)
# X_train=np.array(X_train)
# X_valid=np.array(X_valid)
X_train

<5567x33145 sparse matrix of type '<class 'numpy.int64'>'
	with 131735 stored elements in Compressed Sparse Row format>

## [2.2] Train model

In [29]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

## [2.3] Evaluation

In [30]:
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.5804195804195804


في المثال التالي طريقة إدخال وكتابة اسم النموذج والعمليات وإدخال الدقة

In [31]:
model_comparison_table['model_name'].append("logistic regression with bag of words")
model_comparison_table['preprocessing_methods'].append("none")
model_comparison_table['accuracy'].append(score)
model_comparison_table

{'accuracy': [0.5804195804195804],
 'model_name': ['logistic regression with bag of words'],
 'preprocessing_methods': ['none']}

# Step [3]: Build model with preprocessing methods

## Step[3.1] removing mentions and urls

In [32]:
train_after_pro = preprocess(train1,[1,0,0,0,0,0,0,0])
vectorizer = CountVectorizer()
vectorizer.fit(train_after_pro)

X_train = vectorizer.transform(train_after_pro)
X_valid = vectorizer.transform(valid1)
X_test  = vectorizer.transform(test1)
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.5782112624217888


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [33]:
model_comparison_table['model_name'].append("logistic regression with bag of words")
model_comparison_table['preprocessing_methods'].append("removing mentions and urls")
model_comparison_table['accuracy'].append(score)
model_comparison_table

{'accuracy': [0.5804195804195804, 0.5782112624217888],
 'model_name': ['logistic regression with bag of words',
  'logistic regression with bag of words'],
 'preprocessing_methods': ['none', 'removing mentions and urls']}

## Step[3.2] removing deprecated letters from a word

In [34]:
train_after_pro = preprocess(train1,[0,1,0,0,0,0,0,0])
vectorizer = CountVectorizer()
vectorizer.fit(train_after_pro)

X_train = vectorizer.transform(train_after_pro)
X_valid = vectorizer.transform(valid1)
X_test  = vectorizer.transform(test1)
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.5940375414059624


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [35]:
model_comparison_table['model_name'].append("logistic regression with bag of words")
model_comparison_table['preprocessing_methods'].append("removing deprecated letters from a word")
model_comparison_table['accuracy'].append(score)
model_comparison_table

{'accuracy': [0.5804195804195804, 0.5782112624217888, 0.5940375414059624],
 'model_name': ['logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words'],
 'preprocessing_methods': ['none',
  'removing mentions and urls',
  'removing deprecated letters from a word']}

## Step[3.3] removing digits

In [36]:
train_after_pro = preprocess(train1,[0,0,1,0,0,0,0,0])
vectorizer = CountVectorizer()
vectorizer.fit(train_after_pro)

X_train = vectorizer.transform(train_after_pro)
X_valid = vectorizer.transform(valid1)
X_test  = vectorizer.transform(test1)
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.5807876334192124


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [37]:
model_comparison_table['model_name'].append("logistic regression with bag of words")
model_comparison_table['preprocessing_methods'].append("removing digits")
model_comparison_table['accuracy'].append(score)
model_comparison_table

{'accuracy': [0.5804195804195804,
  0.5782112624217888,
  0.5940375414059624,
  0.5807876334192124],
 'model_name': ['logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words'],
 'preprocessing_methods': ['none',
  'removing mentions and urls',
  'removing deprecated letters from a word',
  'removing digits']}

## Step[3.4] removing non characters

In [38]:
train_after_pro = preprocess(train1,[0,0,0,1,0,0,0,0])
vectorizer = CountVectorizer()
vectorizer.fit(train_after_pro)

X_train = vectorizer.transform(train_after_pro)
X_valid = vectorizer.transform(valid1)
X_test  = vectorizer.transform(test1)
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.5774751564225249


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [39]:
model_comparison_table['model_name'].append("logistic regression with bag of words")
model_comparison_table['preprocessing_methods'].append("removing non characters")
model_comparison_table['accuracy'].append(score)
model_comparison_table

{'accuracy': [0.5804195804195804,
  0.5782112624217888,
  0.5940375414059624,
  0.5807876334192124,
  0.5774751564225249],
 'model_name': ['logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words'],
 'preprocessing_methods': ['none',
  'removing mentions and urls',
  'removing deprecated letters from a word',
  'removing digits',
  'removing non characters']}

## Step[3.5] removing emojies

In [40]:
train_after_pro = preprocess(train1,[0,0,0,0,1,0,0,0])
vectorizer = CountVectorizer()
vectorizer.fit(train_after_pro)

X_train = vectorizer.transform(train_after_pro)
X_valid = vectorizer.transform(valid1)
X_test  = vectorizer.transform(test1)
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.5785793154214207


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [41]:
model_comparison_table['model_name'].append("logistic regression with bag of words")
model_comparison_table['preprocessing_methods'].append("removing emojies")
model_comparison_table['accuracy'].append(score)
model_comparison_table

{'accuracy': [0.5804195804195804,
  0.5782112624217888,
  0.5940375414059624,
  0.5807876334192124,
  0.5774751564225249,
  0.5785793154214207],
 'model_name': ['logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words'],
 'preprocessing_methods': ['none',
  'removing mentions and urls',
  'removing deprecated letters from a word',
  'removing digits',
  'removing non characters',
  'removing emojies']}

## Step[3.6] removing stop word

In [42]:
train_after_pro = preprocess(train1,[0,0,0,0,0,1,0,0])
vectorizer = CountVectorizer()
vectorizer.fit(train_after_pro)

X_train = vectorizer.transform(train_after_pro)
X_valid = vectorizer.transform(valid1)
X_test  = vectorizer.transform(test1)
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.4994479205005521


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [43]:
model_comparison_table['model_name'].append("logistic regression with bag of words")
model_comparison_table['preprocessing_methods'].append("removing stop word")
model_comparison_table['accuracy'].append(score)
model_comparison_table

{'accuracy': [0.5804195804195804,
  0.5782112624217888,
  0.5940375414059624,
  0.5807876334192124,
  0.5774751564225249,
  0.5785793154214207,
  0.4994479205005521],
 'model_name': ['logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words'],
 'preprocessing_methods': ['none',
  'removing mentions and urls',
  'removing deprecated letters from a word',
  'removing digits',
  'removing non characters',
  'removing emojies',
  'removing stop word']}

## Step[3.7] use character standardization

In [44]:
train_after_pro = preprocess(train1,[0,0,0,0,0,0,1,0])
vectorizer = CountVectorizer()
vectorizer.fit(train_after_pro)

X_train = vectorizer.transform(train_after_pro)
X_valid = vectorizer.transform(valid1)
X_test  = vectorizer.transform(test1)
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.5311004784688995


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [45]:
model_comparison_table['model_name'].append("logistic regression with bag of words")
model_comparison_table['preprocessing_methods'].append("character standardization")
model_comparison_table['accuracy'].append(score)
model_comparison_table

{'accuracy': [0.5804195804195804,
  0.5782112624217888,
  0.5940375414059624,
  0.5807876334192124,
  0.5774751564225249,
  0.5785793154214207,
  0.4994479205005521,
  0.5311004784688995],
 'model_name': ['logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words'],
 'preprocessing_methods': ['none',
  'removing mentions and urls',
  'removing deprecated letters from a word',
  'removing digits',
  'removing non characters',
  'removing emojies',
  'removing stop word',
  'character standardization']}

## Step[3.8] stemming

In [46]:
train_after_pro = preprocess(train1,[0,0,0,0,0,0,0,1])
vectorizer = CountVectorizer()
vectorizer.fit(train_after_pro)

X_train = vectorizer.transform(train_after_pro)
X_valid = vectorizer.transform(valid1)
X_test  = vectorizer.transform(test1)
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.5811556864188443


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [47]:
model_comparison_table['model_name'].append("logistic regression with bag of words")
model_comparison_table['preprocessing_methods'].append("after stemming")
model_comparison_table['accuracy'].append(score)
model_comparison_table

{'accuracy': [0.5804195804195804,
  0.5782112624217888,
  0.5940375414059624,
  0.5807876334192124,
  0.5774751564225249,
  0.5785793154214207,
  0.4994479205005521,
  0.5311004784688995,
  0.5811556864188443],
 'model_name': ['logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words'],
 'preprocessing_methods': ['none',
  'removing mentions and urls',
  'removing deprecated letters from a word',
  'removing digits',
  'removing non characters',
  'removing emojies',
  'removing stop word',
  'character standardization',
  'after stemming']}

## Step[3.9] removing deprecated letters from a word,removing digits,stemming

In [48]:
train_after_pro = preprocess(train1,[0,1,1,0,0,0,0,0,1])
# X_test=np.array(test1)
# train_after_pro=np.array(train_after_pro)
# X_valid=np.array(valid1)

vectorizer = CountVectorizer()
vectorizer.fit(train_after_pro)

X_train = vectorizer.transform(train_after_pro)
X_valid = vectorizer.transform(valid1)
X_test  = vectorizer.transform(test1)
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.5925653294074347


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [49]:
model_comparison_table['model_name'].append("Logistic regression")
model_comparison_table['preprocessing_methods'].append("removing deprecated letters from a word,removing digits,stemming")
model_comparison_table['accuracy'].append(score)
model_comparison_table

{'accuracy': [0.5804195804195804,
  0.5782112624217888,
  0.5940375414059624,
  0.5807876334192124,
  0.5774751564225249,
  0.5785793154214207,
  0.4994479205005521,
  0.5311004784688995,
  0.5811556864188443,
  0.5925653294074347],
 'model_name': ['logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'Logistic regression'],
 'preprocessing_methods': ['none',
  'removing mentions and urls',
  'removing deprecated letters from a word',
  'removing digits',
  'removing non characters',
  'removing emojies',
  'removing stop word',
  'character standardization',
  'after stemming',
  'removing deprecated letters from a word,removing digits,stemming']}

# Logistic Regression with TF-IDF

In [50]:
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer(min_df=1, max_df=0.10, ngram_range=(1, 4))

    train = tfidf_vectorizer.fit_transform(data)

    return train, tfidf_vectorizer


train_after_pro = preprocess(train1,[0,1,1,0,0,0,0,0,1])
X_train_tfidf, tfidf_vectorizer = tfidf(train_after_pro)
X_test_tfidf = tfidf_vectorizer.transform(test1)
X_valid_tfidf = tfidf_vectorizer.transform(valid1)


In [51]:
clf = LogisticRegression(random_state=40)
clf.fit(X_train_tfidf, y_train)

y_predicted = clf.predict(X_test_tfidf)

In [52]:
# accuracy_score(y_valid, y_predicted)
score = clf.score(X_test_tfidf,y_test)

In [53]:
model_comparison_table['model_name'].append("tfidf with logistic regression")
model_comparison_table['preprocessing_methods'].append("removing deprecated letters from a word,removing digits,stemming")
model_comparison_table['accuracy'].append(score)
model_comparison_table

{'accuracy': [0.5804195804195804,
  0.5782112624217888,
  0.5940375414059624,
  0.5807876334192124,
  0.5774751564225249,
  0.5785793154214207,
  0.4994479205005521,
  0.5311004784688995,
  0.5811556864188443,
  0.5925653294074347,
  0.6209054103790946],
 'model_name': ['logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'Logistic regression',
  'tfidf with logistic regression'],
 'preprocessing_methods': ['none',
  'removing mentions and urls',
  'removing deprecated letters from a word',
  'removing digits',
  'removing non characters',
  'removing emojies',
  'removing stop word',
  'character standardization',
  'after stemming',
  'removing deprecated 

# MultinomialNB with Tfidf

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline


text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('RUS', RandomOverSampler()),
                    ('clf', MultinomialNB())])
text_clf = text_clf.fit(train1, y_train)
y_pred = text_clf.predict(test1)
score = text_clf.score(test1, y_test)
print('Score:',text_clf.score(test1, y_test))

Score: 0.6345233713654767


In [55]:
model_comparison_table['model_name'].append("MultinomialNB with TfidfTransformer")
model_comparison_table['preprocessing_methods'].append("None")
model_comparison_table['accuracy'].append(score)
model_comparison_table

{'accuracy': [0.5804195804195804,
  0.5782112624217888,
  0.5940375414059624,
  0.5807876334192124,
  0.5774751564225249,
  0.5785793154214207,
  0.4994479205005521,
  0.5311004784688995,
  0.5811556864188443,
  0.5925653294074347,
  0.6209054103790946,
  0.6345233713654767],
 'model_name': ['logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'Logistic regression',
  'tfidf with logistic regression',
  'MultinomialNB with TfidfTransformer'],
 'preprocessing_methods': ['none',
  'removing mentions and urls',
  'removing deprecated letters from a word',
  'removing digits',
  'removing non characters',
  'removing emojies',
  'removing stop word',
  'characte

# Keras

In [56]:
vectorizer = CountVectorizer()
vectorizer.fit(train1)

X_train = vectorizer.transform(train1)
X_valid = vectorizer.transform(valid1)
X_test  = vectorizer.transform(test1)
y_test=np.array(y_test)
y_train=np.array(y_train)
y_valid=np.array(y_valid)

In [57]:
import tensorflow as tf
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)
print(type(X_train_tfidf))

X_train_tfidf=convert_sparse_matrix_to_sparse_tensor(X_train_tfidf)
X_train_tfidf = tf.sparse.reorder(X_train_tfidf)

X_test_tfidf=convert_sparse_matrix_to_sparse_tensor(X_test_tfidf)
X_test_tfidf = tf.sparse.reorder(X_test_tfidf)

X_valid_tfidf = convert_sparse_matrix_to_sparse_tensor(X_valid_tfidf)
X_valid_tfidf = tf.sparse.reorder(X_valid_tfidf)

<class 'scipy.sparse.csr.csr_matrix'>


## Model with tfidf data

In [58]:
from keras.models import Sequential
from keras import layers

input_dim = X_train_tfidf.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [59]:
model.compile(loss='binary_crossentropy', 
               optimizer='adam', 
               metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                3491000   
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 3,491,011
Trainable params: 3,491,011
Non-trainable params: 0
_________________________________________________________________


In [60]:
history = model.fit(X_train_tfidf, y_train,
                     epochs=30,
                     verbose=True,
                     validation_data=(X_valid_tfidf, y_valid),
                     batch_size=20)

Epoch 1/30


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:450: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 10), dtype=float32), dense_shape=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


279/279 [==============================] - 11s 37ms/step - loss: 0.6927 - accuracy: 0.5121 - val_loss: 0.6864 - val_accuracy: 0.5986
Epoch 2/30
279/279 [==============================] - 10s 36ms/step - loss: 0.5892 - accuracy: 0.9102 - val_loss: 0.6449 - val_accuracy: 0.6510
Epoch 3/30
279/279 [==============================] - 11s 38ms/step - loss: 0.3173 - accuracy: 0.9634 - val_loss: 0.6527 - val_accuracy: 0.6328
Epoch 4/30
279/279 [==============================] - 10s 36ms/step - loss: 0.1481 - accuracy: 0.9732 - val_loss: 0.6805 - val_accuracy: 0.6250
Epoch 5/30
279/279 [==============================] - 10s 36ms/step - loss: 0.0866 - accuracy: 0.9777 - val_loss: 0.7019 - val_accuracy: 0.6250
Epoch 6/30
279/279 [==============================] - 10s 36ms/step - loss: 0.0617 - accuracy: 0.9806 - val_loss: 0.7085 - val_accuracy: 0.6310
Epoch 7/30
279/279 [==============================] - 11s 41ms/step - loss: 0.0482 - accuracy: 0.9828 - val_loss: 0.7513 - val_accuracy: 0.6116
Epo

In [61]:
loss, accuracy = model.evaluate(X_train_tfidf, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test_tfidf, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9943
Testing Accuracy:  0.5944


In [62]:
model_comparison_table['model_name'].append("deep nueral network with tfidf")
model_comparison_table['preprocessing_methods'].append("removing deprecated letters from a word,removing digits,stemming")
model_comparison_table['accuracy'].append(accuracy)
model_comparison_table

{'accuracy': [0.5804195804195804,
  0.5782112624217888,
  0.5940375414059624,
  0.5807876334192124,
  0.5774751564225249,
  0.5785793154214207,
  0.4994479205005521,
  0.5311004784688995,
  0.5811556864188443,
  0.5925653294074347,
  0.6209054103790946,
  0.6345233713654767,
  0.5944055914878845],
 'model_name': ['logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'Logistic regression',
  'tfidf with logistic regression',
  'MultinomialNB with TfidfTransformer',
  'deep nueral network with tfidf'],
 'preprocessing_methods': ['none',
  'removing mentions and urls',
  'removing deprecated letters from a word',
  'removing digits',
  'removing non characters',

### create a tokenizer and adding a padding to create an embedding layer

In [64]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(preprocess(train1,[0,1,1,0,0,0,0,0,1]))

X_train = tokenizer.texts_to_sequences(preprocess(train1,[0,1,1,0,0,0,0,0,1]))
X_test = tokenizer.texts_to_sequences(test1)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

In [65]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


## Model with Embedding

In [66]:
embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 50)           1699350   
                                                                 
 flatten (Flatten)           (None, 5000)              0         
                                                                 
 dense_2 (Dense)             (None, 10)                50010     
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,749,371
Trainable params: 1,749,371
Non-trainable params: 0
_________________________________________________________________


In [67]:
history = model.fit(X_train, y_train,
                    epochs=30,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=20)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
# plot_history(history)

Epoch 1/30
279/279 [==============================] - 6s 20ms/step - loss: 0.6935 - accuracy: 0.4988 - val_loss: 0.6936 - val_accuracy: 0.3342
Epoch 2/30
279/279 [==============================] - 6s 20ms/step - loss: 0.6932 - accuracy: 0.5087 - val_loss: 0.6840 - val_accuracy: 0.6654
Epoch 3/30
279/279 [==============================] - 5s 19ms/step - loss: 0.6329 - accuracy: 0.6364 - val_loss: 0.6752 - val_accuracy: 0.5904
Epoch 4/30
279/279 [==============================] - 5s 19ms/step - loss: 0.2821 - accuracy: 0.9048 - val_loss: 0.8375 - val_accuracy: 0.5351
Epoch 5/30
279/279 [==============================] - 5s 20ms/step - loss: 0.1518 - accuracy: 0.9506 - val_loss: 0.9527 - val_accuracy: 0.5237
Epoch 6/30
279/279 [==============================] - 6s 20ms/step - loss: 0.0992 - accuracy: 0.9653 - val_loss: 1.0040 - val_accuracy: 0.5480
Epoch 7/30
279/279 [==============================] - 6s 20ms/step - loss: 0.0771 - accuracy: 0.9702 - val_loss: 1.0260 - val_accuracy: 0.5591

In [68]:
model_comparison_table['model_name'].append("Embedding")
model_comparison_table['preprocessing_methods'].append("removing deprecated letters from a word,removing digits,stemming")
model_comparison_table['accuracy'].append(accuracy)
model_comparison_table

{'accuracy': [0.5804195804195804,
  0.5782112624217888,
  0.5940375414059624,
  0.5807876334192124,
  0.5774751564225249,
  0.5785793154214207,
  0.4994479205005521,
  0.5311004784688995,
  0.5811556864188443,
  0.5925653294074347,
  0.6209054103790946,
  0.6345233713654767,
  0.5944055914878845,
  0.559808611869812],
 'model_name': ['logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'Logistic regression',
  'tfidf with logistic regression',
  'MultinomialNB with TfidfTransformer',
  'deep nueral network with tfidf',
  'Embedding'],
 'preprocessing_methods': ['none',
  'removing mentions and urls',
  'removing deprecated letters from a word',
  'removing d

## Add a GlobalMaxPool1D layer to the previous model

In [69]:
embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 50)           1699350   
                                                                 
 global_max_pooling1d (Globa  (None, 50)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_4 (Dense)             (None, 10)                510       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,699,871
Trainable params: 1,699,871
Non-trainable params: 0
_________________________________________________________________


In [70]:
history = model.fit(X_train, y_train,
                    epochs=30,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=20)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
# plot_history(history)

Epoch 1/30
279/279 [==============================] - 6s 20ms/step - loss: 0.6926 - accuracy: 0.5166 - val_loss: 0.6766 - val_accuracy: 0.6905
Epoch 2/30
279/279 [==============================] - 6s 20ms/step - loss: 0.6704 - accuracy: 0.6397 - val_loss: 0.6617 - val_accuracy: 0.6367
Epoch 3/30
279/279 [==============================] - 6s 20ms/step - loss: 0.5905 - accuracy: 0.7408 - val_loss: 0.6738 - val_accuracy: 0.5889
Epoch 4/30
279/279 [==============================] - 6s 20ms/step - loss: 0.4481 - accuracy: 0.8197 - val_loss: 0.7430 - val_accuracy: 0.5664
Epoch 5/30
279/279 [==============================] - 6s 20ms/step - loss: 0.3142 - accuracy: 0.8911 - val_loss: 0.8182 - val_accuracy: 0.5756
Epoch 6/30
279/279 [==============================] - 5s 19ms/step - loss: 0.2166 - accuracy: 0.9294 - val_loss: 0.9031 - val_accuracy: 0.5793
Epoch 7/30
279/279 [==============================] - 5s 20ms/step - loss: 0.1543 - accuracy: 0.9520 - val_loss: 1.0411 - val_accuracy: 0.5731

In [71]:
model_comparison_table['model_name'].append("Embedding with maxbooling")
model_comparison_table['preprocessing_methods'].append("removing deprecated letters from a word,removing digits,stemming")
model_comparison_table['accuracy'].append(accuracy)

## Add a CNN layer to the previous model

In [72]:
embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 100)          3398700   
                                                                 
 conv1d (Conv1D)             (None, 96, 128)           64128     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_6 (Dense)             (None, 10)                1290      
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
Total params: 3,464,129
Trainable params: 3,464,129
Non-trainable params: 0
____________________________________________

In [73]:
history = model.fit(X_train, y_train,
                    epochs=30,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=20)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
# plot_history(history)

Epoch 1/30
279/279 [==============================] - 18s 58ms/step - loss: 0.6934 - accuracy: 0.5019 - val_loss: 0.6957 - val_accuracy: 0.3485
Epoch 2/30
279/279 [==============================] - 16s 57ms/step - loss: 0.6773 - accuracy: 0.5847 - val_loss: 0.6474 - val_accuracy: 0.6272
Epoch 3/30
279/279 [==============================] - 16s 56ms/step - loss: 0.5035 - accuracy: 0.7746 - val_loss: 0.7257 - val_accuracy: 0.5709
Epoch 4/30
279/279 [==============================] - 16s 56ms/step - loss: 0.2444 - accuracy: 0.9163 - val_loss: 0.8519 - val_accuracy: 0.5760
Epoch 5/30
279/279 [==============================] - 17s 60ms/step - loss: 0.1415 - accuracy: 0.9549 - val_loss: 0.9954 - val_accuracy: 0.5742
Epoch 6/30
279/279 [==============================] - 16s 57ms/step - loss: 0.1152 - accuracy: 0.9659 - val_loss: 1.0524 - val_accuracy: 0.5657
Epoch 7/30
279/279 [==============================] - 16s 59ms/step - loss: 0.0896 - accuracy: 0.9709 - val_loss: 1.0623 - val_accuracy:

In [74]:
model_comparison_table['model_name'].append("CNN with Embedding")
model_comparison_table['preprocessing_methods'].append("removing deprecated letters from a word,removing digits,stemming")
model_comparison_table['accuracy'].append(accuracy)
model_comparison_table

{'accuracy': [0.5804195804195804,
  0.5782112624217888,
  0.5940375414059624,
  0.5807876334192124,
  0.5774751564225249,
  0.5785793154214207,
  0.4994479205005521,
  0.5311004784688995,
  0.5811556864188443,
  0.5925653294074347,
  0.6209054103790946,
  0.6345233713654767,
  0.5944055914878845,
  0.559808611869812,
  0.5675377249717712,
  0.5060728788375854],
 'model_name': ['logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'logistic regression with bag of words',
  'Logistic regression',
  'tfidf with logistic regression',
  'MultinomialNB with TfidfTransformer',
  'deep nueral network with tfidf',
  'Embedding',
  'Embedding with maxbooling',
  'CNN with Embedding'],
 'preprocessing_methods': 

## [3.3] Evaluation

In [75]:
import pandas as pd

df = pd.DataFrame(model_comparison_table)
df

,model_name,preprocessing_methods,accuracy
0,logistic regression with bag of words,none,0.580420
1,logistic regression with bag of words,removing mentions and urls,0.578211
2,logistic regression with bag of words,removing deprecated letters from a word,0.594038
3,logistic regression with bag of words,removing digits,0.580788
4,logistic regression with bag of words,removing non characters,0.577475
5,logistic regression with bag of words,removing emojies,0.578579
6,logistic regression with bag of words,removing stop word,0.499448
7,logistic regression with bag of words,character standardization,0.531100
8,logistic regression with bag of words,after stemming,0.581156
9,Logistic regression,"removing deprecated letters from a word,removi...",0.592565
